In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_image='D:/Documents/Capstone/Project -2/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)

In [4]:
training_set=train_datagen.flow_from_directory(train_image,
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 47454 images belonging to 35 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:/Documents/Capstone/Project -2/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 14321 images belonging to 35 classes.


In [7]:
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(224,224,3)
                     )
 
base_model.trainable=False
 
model = tf.keras.Sequential([
        base_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(35, activation='softmax')
    ])

In [8]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 batch_normalization_407 (Ba  (None, 5, 5, 1536)       10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d_1   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 512)               786944    
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                      

In [10]:
early = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

In [11]:
batch_size=32
STEP_SIZE_TRAIN = training_set.n//training_set.batch_size
STEP_SIZE_VALID = test_set.n//test_set.batch_size

# fit model
history = model.fit(training_set, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=15,
                    callbacks=[early])

Epoch 1/15
9490/9490 [==============================] - 28004s 3s/step - loss: 0.8918 - accuracy: 0.7432 - val_loss: 5.9932 - val_accuracy: 0.5573
Epoch 2/15
9490/9490 [==============================] - 25085s 3s/step - loss: 0.4796 - accuracy: 0.8664 - val_loss: 8.2227 - val_accuracy: 0.6133
Epoch 3/15
9490/9490 [==============================] - 26762s 3s/step - loss: 0.4094 - accuracy: 0.8905 - val_loss: 9.6019 - val_accuracy: 0.6324
Epoch 4/15
9490/9490 [==============================] - 28101s 3s/step - loss: 0.3681 - accuracy: 0.9005 - val_loss: 14.6063 - val_accuracy: 0.6277
Epoch 5/15
9490/9490 [==============================] - 26100s 3s/step - loss: 0.3630 - accuracy: 0.9060 - val_loss: 17.7141 - val_accuracy: 0.6348
Epoch 6/15
9490/9490 [==============================] - 25905s 3s/step - loss: 0.3421 - accuracy: 0.9119 - val_loss: 16.0180 - val_accuracy: 0.6422
Epoch 7/15
9490/9490 [==============================] - 25427s 3s/step - loss: 0.3256 - accuracy: 0.9188 - val_loss

KeyboardInterrupt: 